In [1]:
!sudo pip install vertica_python[namedparams]

The directory '/home/developer/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/developer/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.


Boilerplate Connection Setup
---

In [2]:
%load_ext edx_jupyter

In [3]:
%reload_ext edx_jupyter

In [4]:
%%vertica

SELECT 'Connected'

password: ········


,?column?
0,Connected


Set up parameters
---

In [5]:
less_recent_than = 7
more_recent_than = 31

Store queries
---

In [6]:
find_disengaged_users = """
SELECT
    CASE
        WHEN act.course_start_date >= uc.first_enrollment_time THEN act.course_start_date
        WHEN act.course_start_date < uc.first_enrollment_time THEN DATE(uc.first_enrollment_time)
    END AS content_availability_date,
    act.user_id,
    act.course_id,
    act.last_activity_date
FROM
(
    SELECT
        act.*,
        crs.course_start_date
    FROM
    (
        SELECT
            user_id,
            course_id,
            MAX(date) AS last_activity_date
        FROM
            business_intelligence.activity_engagement_user_daily
        WHERE
                is_engaged
            AND date <= :end_date
        GROUP BY 1, 2
    ) act
    JOIN
            experiment_courses as crs
        ON act.course_id = crs.course_id
    WHERE
        DATEDIFF('day', act.last_activity_date, :end_date) BETWEEN :less_recent_than AND :more_recent_than
) as act
JOIN
        production.d_user_course as uc
    ON uc.user_id = act.user_id
    AND uc.course_id = act.course_id
    AND (uc.first_verified_enrollment_time is NULL
         OR uc.first_verified_enrollment_time > :end_date)
"""
    
latest_disengaged_course = """
SELECT DISTINCT
    user_id,
    LAST_VALUE(course_id) OVER (
        PARTITION BY user_id ORDER BY last_activity_date
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
    ) as course_id,
    LAST_VALUE(last_activity_date) OVER (
        PARTITION BY user_id ORDER BY last_activity_date
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
    ) as last_activity_date
"""

Find the most recent date of user activity imported
---

In [7]:
%%vertica -> latest_activity
SELECT MAX(date) FROM production.f_user_activity

In [8]:
end_date = latest_activity['MAX'][0]

Find the courses that meet our criteria
---

In [9]:
%%vertica (end_date=end_date)
DROP TABLE IF EXISTS experiment_courses;

CREATE TEMPORARY TABLE experiment_courses
ON COMMIT PRESERVE ROWS
AS SELECT
    course_id,
    course_start_date
FROM
    business_intelligence.course_master
WHERE
        (content_language is NULL OR content_language = 'en-us')
    AND pacing_type = 'self_paced'
    AND DATEDIFF('day', :end_date, course_verification_end_date) > 7
    AND :end_date < course_end_date
    AND DATEDIFF('day', course_start_date, :end_date) > 7

,MAX


In [10]:
%vertica SELECT COUNT(*) FROM experiment_courses

,COUNT
0,335


Find recently disengaged users
---

In [11]:
%%vertica (end_date=end_date, less_recent_than=less_recent_than, more_recent_than=more_recent_than)
DROP TABLE IF EXISTS recently_disengaged_users;

CREATE TEMPORARY TABLE recently_disengaged_users
ON COMMIT PRESERVE ROWS
AS {find_disengaged_users}

,COUNT


In [12]:
%vertica SELECT COUNT(*) FROM recently_disengaged_users

,COUNT
0,230805


In [13]:
%%vertica
DROP TABLE IF EXISTS most_recently_disengaged_user_course;

CREATE TEMPORARY TABLE most_recently_disengaged_user_course
ON COMMIT PRESERVE ROWS
AS {latest_disengaged_course} FROM recently_disengaged_users

,COUNT


In [14]:
%vertica SELECT COUNT(*) FROM most_recently_disengaged_user_course

,COUNT
0,187876


Count unique recently disengaged users in self-paced courses
---

In [15]:
%%vertica

SELECT
    COUNT(DISTINCT user_id)
FROM
    most_recently_disengaged_user_course

,COUNT
0,187876


Count users disengaged from multiple courses
---

In [16]:
%%vertica

SELECT
    disengaged_count,
    COUNT(DISTINCT user_id)
FROM
(
    SELECT
        user_id,
        COUNT(DISTINCT course_id) as disengaged_count
    FROM
        recently_disengaged_users
    GROUP BY
        1
    HAVING
        COUNT(DISTINCT course_id) > 1
) as cnt
GROUP BY
    1
ORDER BY 1

,disengaged_count,COUNT
0,2,21292
1,3,5082
2,4,1649
3,5,591
4,6,294
5,7,140
6,8,73
7,9,48
8,10,31
9,11,13


Count unique recently disengaged users by amount of activity
---

In [17]:
%%vertica

SELECT
    act.activity_count,
    COUNT(DISTINCT act.user_id)
FROM
(
    SELECT
        act.user_id,
        act.course_id,
        count(*) as activity_count
    FROM
        production.f_user_activity as act
    JOIN
            most_recently_disengaged_user_course as rdu
        ON rdu.user_id = act.user_id
        AND rdu.course_id = act.course_id
    WHERE
            activity_type != 'ACTIVE'
        AND DATEDIFF('day', date, '{end_date}') BETWEEN {less_recent_than} AND {more_recent_than}
    GROUP BY 1, 2
) act
GROUP BY
    1
ORDER BY
    1

,activity_count,COUNT
0,1,76889
1,2,45452
2,3,17090
3,4,13618
4,5,7505
5,6,6577
6,7,4032
7,8,3582
8,9,2437
9,10,2127


VTR for English self-paced courses that started within the last year
---

In [18]:
%%vertica

SELECT
    SUM(sum_verifications) / SUM(sum_enrolls_vtr)
FROM
    business_intelligence.course_stats_summary as stats
JOIN
        production.d_course as crs
    ON stats.course_id = crs.course_id
WHERE
        (crs.content_language is NULL OR crs.content_language = 'en-us')
    AND crs.pacing_type = 'self_paced'
    AND crs.start_time >= '2016-06-01'

,?column?
0,0.019722636313664237


Count of unique recently disengaged users that have > 50% grade
---

In [19]:
%%vertica

SELECT
    (grd.percent_grade > 0.5),
    COUNT(DISTINCT rdu.user_id)
FROM
    most_recently_disengaged_user_course as rdu
JOIN
(
    SELECT
        course_id,
        user_id,
        MAX(percent_grade) as percent_grade
    FROM
        business_intelligence.grade_user_time
    GROUP BY
        1, 2
) as grd
    ON rdu.course_id = grd.course_id
   AND rdu.user_id = grd.user_id
GROUP BY
    1
ORDER BY
    1 DESC


,?column?,COUNT
0,True,13620
1,False,128221


Count of recently disengaged users by country
---

In [20]:
%%vertica

SELECT
    usr.user_last_location_country_code,
    COUNT(DISTINCT rdu.user_id)
FROM
    most_recently_disengaged_user_course as rdu
JOIN
        production.d_user as usr
    ON rdu.user_id = usr.user_id
GROUP BY
    1
ORDER BY
    2 DESC

,user_last_location_country_code,COUNT
0,US,46802
1,IN,21160
2,UNKNOWN,9507
3,CA,8930
4,GB,6768
5,BR,5095
6,AU,4211
7,MX,4027
8,DE,3947
9,ES,3094


Find "recently disengaged" users from last month
---

In [21]:
import datetime

retrospective_end_date = datetime.date(2017, 4, 30)

In [22]:
%%vertica (end_date=retrospective_end_date)
DROP TABLE IF EXISTS experiment_courses;

CREATE TEMPORARY TABLE experiment_courses
ON COMMIT PRESERVE ROWS
AS SELECT
    course_id,
    course_start_date
FROM
    business_intelligence.course_master
WHERE
        (content_language is NULL OR content_language = 'en-us')
    AND pacing_type = 'self_paced'
    AND DATEDIFF('day', :end_date, course_verification_end_date) > 7
    AND :end_date < course_end_date
    AND DATEDIFF('day', course_start_date, :end_date) > 7

,user_last_location_country_code,COUNT


In [23]:
%%vertica
SELECT COUNT(*) FROM experiment_courses;

,COUNT
0,329


In [24]:
%%vertica (end_date=retrospective_end_date, less_recent_than=less_recent_than, more_recent_than=more_recent_than)
DROP TABLE IF EXISTS baseline_recently_disengaged_users;

CREATE TEMPORARY TABLE baseline_recently_disengaged_users
ON COMMIT PRESERVE ROWS
AS {find_disengaged_users}

,COUNT


In [25]:
%%vertica
DROP TABLE IF EXISTS baseline_most_recently_disengaged_user_course;

CREATE TEMPORARY TABLE baseline_most_recently_disengaged_user_course
ON COMMIT PRESERVE ROWS
AS {latest_disengaged_course} FROM baseline_recently_disengaged_users

,COUNT


In [26]:
%%vertica

SELECT COUNT(DISTINCT user_id)
FROM
    baseline_most_recently_disengaged_user_course

,COUNT
0,168356


Compute the return rate of recently disengaged users from last month
---

In [27]:
%%vertica -> return_rates
SELECT
    returned_after,
    COUNT(DISTINCT user_id)
FROM
(
    SELECT
        rdu.user_id,
        DATEDIFF('day', '{retrospective_end_date}', MIN(act.date)) as returned_after
    FROM
        baseline_most_recently_disengaged_user_course as rdu
    LEFT JOIN
            production.f_user_activity as act
         ON act.user_id = rdu.user_id
        AND act.course_id = rdu.course_id
        AND act.date >= '{retrospective_end_date}'
        AND act.activity_type != 'ACTIVE'
    GROUP BY
        1
) as return
GROUP BY
    1
ORDER BY 1

In [28]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import pandas as pd
import numpy as np
init_notebook_mode(connected=True)

In [29]:
total_users = return_rates['COUNT'].sum()

In [30]:
iplot([{
    "x": return_rates['returned_after'][1:],
    "y": return_rates['COUNT'][1:].cumsum()/total_users*100
}])

Compute the bookings rate of recently disengaged users from last month
---

In [31]:
%%vertica -> verify_rates
SELECT
    verified_after,
    COUNT(DISTINCT user_id)
FROM
(
    SELECT
        rdu.user_id,
        DATEDIFF('day', '{retrospective_end_date}', MIN(crs.first_verified_enrollment_time)) as verified_after
    FROM
        baseline_most_recently_disengaged_user_course as rdu
    LEFT JOIN
            production.d_user_course as crs
        ON rdu.user_id = crs.user_id
        AND rdu.course_id = crs.course_id
    GROUP BY
        1
) as return
GROUP BY
    1
ORDER BY 1

In [32]:
verify_rates

,verified_after,COUNT
0,NaN,167693
1,0.0,3
2,1.0,25
3,2.0,15
4,3.0,18
5,4.0,23
6,5.0,27
7,6.0,20
8,7.0,15
9,8.0,34


In [33]:
total_unverified_users = verify_rates['COUNT'].sum()

In [34]:
iplot([{
    "x": verify_rates['verified_after'][1:],
    "y": verify_rates['COUNT'][1:].cumsum()/total_unverified_users*100
}])

Effective VTR for disengaged users that return to the site
---

In [35]:
iplot([{
    "x": verify_rates['verified_after'][1:],
    "y": verify_rates['COUNT'][1:].cumsum()/return_rates['COUNT'][1:].cumsum()*100
}])

Measure Time-Since-Last-Engagement for recently-disengaged users
---

In [36]:
%%vertica (end_date=retrospective_end_date) -> time_since_last_activity

SELECT
    DATEDIFF('day', last_activity_date, :end_date),
    COUNT(*)
FROM
    baseline_most_recently_disengaged_user_course
GROUP BY
    1
ORDER BY 1

In [37]:
time_since_last_activity['COUNT']

0     7779
1     7297
2     9682
3     9485
4     9059
5     8862
6     7609
7     5273
8     4852
9     5505
10    6543
11    7726
12    7683
13    6355
14    5151
15    4946
16    6260
17    7075
18    7786
19    7738
20    6080
21    4931
22    4500
23    4542
24    5637
Name: COUNT, dtype: int64

In [42]:
iplot([{
    'x': [retrospective_end_date - datetime.timedelta(days=int(t)) for t in time_since_last_activity['datediff']],
    'y': time_since_last_activity['COUNT'],
}])